# NYC Airbnb 數據預處理與清洗
---
**專案目標：** 透過數據清洗與特徵工程，建立高品質的數據集以供後續分析或機器學習模型使用。

### 數據集概覽 (Metadata)
| 類別 | 關鍵欄位 | 說明 |
|---|---|---|
| **房源資訊** | `id`, `name`, `room_type` | 包含房名稱及類型 |
| **房東資訊** | `host_id`, `host_name`, `calculated_host_listings_count` | 房東相關資訊 |
| **地理資訊** | `neighbourhood_group`, `neighbourhood`, `latitude`, `longitude` | 涵蓋紐約五大行政區 |
| **定價與條件** | `price`, `minimum_nights`, `availability_365` | 房源核心經濟指標 |
| **評論指標** |   `reviews_per_month`, `number_of_reviews`, `last_review` | 用戶活躍度指標 |

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import kagglehub
from kagglehub import KaggleDatasetAdapter

# 視覺化風格設定
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

## 1. 數據載入與初步診斷
從資料源獲取原始數據，並檢查缺失值情況。

In [8]:
file_path = "AB_NYC_2019.csv"
df_raw = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "dgomonov/new-york-city-airbnb-open-data",
    file_path
)

# 檢查缺失值比例
null_summary = df_raw.isnull().mean() * 100
print("各欄位缺失值百分比：\n", null_summary[null_summary > 0])

# 檢查重複值
print("重複值筆數：", df_raw.duplicated().sum(), "筆")

各欄位缺失值百分比：
 name                  0.032723
host_name             0.042949
last_review          20.558339
reviews_per_month    20.558339
dtype: float64
重複值筆數： 0 筆


## 2. 數據清洗策略
為了避免數據偏誤，採取以下行動：
1. **填補缺失值**：針對評論相關欄位進行填補。
2. **過濾離群值**：使用分組 IQR 方法處理異常價格。
3. **過濾殭屍房源**：計算房源自最後一次評論以來的時間，過濾不活躍房源。

In [9]:
# A. 處理缺失值
df_proc = df_raw.copy()
df_proc['name'] = df_proc['name'].fillna('unknown')
df_proc['host_name'] = df_proc['host_name'].fillna('unknown')
df_proc['reviews_per_month'] = df_proc['reviews_per_month'].fillna(0)

# B. 分組移除價格離群值 (依據 neighbourhood_group)
group_q1 = df_proc.groupby('neighbourhood_group')['price'].transform(lambda x: x.quantile(0.25))
group_q3 = df_proc.groupby('neighbourhood_group')['price'].transform(lambda x: x.quantile(0.75))

iqr = group_q3 - group_q1
lower = (group_q1 - 1.5 * iqr).clip(lower=0)
upper = group_q3 + 1.5 * iqr

df_filtered = df_proc[(df_proc['price'] >= lower) & (df_proc['price'] <= upper)].copy()

# C. 過濾不活躍房源 (2 年以上無評論)
df_filtered['last_review'] = pd.to_datetime(df_filtered['last_review'])
current_date = df_filtered['last_review'].max()
df_filtered['days_since_review'] = (current_date - df_filtered['last_review']).dt.days
df_active = df_filtered[df_filtered['days_since_review'] < 730].copy()

print(f"數據清洗完成。樣本量從 {len(df_raw):,} 筆，縮減至 {len(df_active):,} 筆。")

數據清洗完成。樣本量從 48,895 筆，縮減至 31,566 筆。


## 3. 數據整合與索引重設
在完成多重過濾後，目前的 DataFrame 索引是斷續且混亂的，需要重設索引以利於後續矩陣運算與數據分析。

In [10]:
# 重設索引：drop=True 避免將舊索引存為新欄位
df_final = df_active.reset_index(drop=True)

# 最終數據檢查
df_final.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,days_since_review
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,262.0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,48.0
2,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,3.0
3,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0,231.0
4,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129,16.0


In [11]:
df_final.to_csv("nyc_airbnb_data.csv")